In [ ]:
%pip install pydot
%pip install graphviz

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
import keras_tuner

from sklearn.model_selection import RandomizedSearchCV, train_test_split

%matplotlib inline

## Preprocess Data

In [ ]:
data = pd.read_csv('full_data.csv')

In [ ]:
headings = data.columns.to_list()
print(len(headings))
headings_without_label = headings.pop(19)
print(len(headings))
headings.append('label')
print(headings)

In [ ]:
data = data[headings]

In [ ]:
data

In [ ]:
# data.drop(columns=data.columns[0], axis=1,  inplace=True)

# data

In [ ]:
shuf_data = data.sample(frac=1)
shuf_data.head()

In [ ]:
no_events = shuf_data.shape[0]
no_training = int(0.7 * no_events)
no_val = int(no_training + (0.15 * no_events))

In [ ]:
dataset = shuf_data.to_numpy()

In [ ]:
X = dataset[:, 0:28]
y = dataset[:, 28]

In [ ]:
# X_train
X_train = X[:no_training]
X_val = X[no_training:no_val]
X_test = X[no_val:]

y_train = y[:no_training]
y_val = y[no_training:no_val]
y_test = y[no_val:]

## Evaluating Hyperparameters

In [ ]:
def build_model(hp):
    model = Sequential()
    
    # tuning number of layers:
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Dense(units=hp.Int(f'units{i}', min_value=10, max_value=80, step=10),
                        activation=hp.Choice('activation', ['relu', 'tanh']),
                       )
                 )
    
    if hp.Boolean('dropout'):
        model.add(Dropout(rate=0.2))
    
    model.add(Dense(1, activation='sigmoid'))
    #Add learning rate?
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    
    return(model)
        

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=20, 
                                 executions_per_trial=1, overwrite=True, directory='./NN_traing_res',
                                 project_name='res_1')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train, epochs=50, batch_size=500,  validation_data=(X_val, y_val))

In [ ]:
tuner.results_summary()

## Define Model With Best Hyperparams

In [ ]:
# define the keras model
# model = Sequential()
# model.add(Dense(50, activation='relu', name='ReLu-1'))
# model.add(Dense(80, activation='relu', name='ReLu-2'))
# model.add(Dense(70, activation='relu', name='ReLu-3'))
# model.add(Dense(80, activation='relu', name='ReLu-4'))
# model.add(Dropout(rate=0.2))
# model.add(Dense(1, activation='sigmoid', name='Sigmoid'))

model = Sequential()
model.add(Dense(70, activation='relu', name='ReLu-1'))
model.add(Dense(60, activation='relu', name='ReLu-2'))
model.add(Dense(50, activation='relu', name='ReLu-3'))
model.add(Dense(10, activation='relu', name='ReLu-4'))
model.add(Dense(1, activation='sigmoid', name='Sigmoid'))

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.000335981718179628), metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0012413505390506882), metrics=['accuracy'])

## Train and Evaluate Model

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=500)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
predictions = model.predict(X_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]

In [ ]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, rankdir="LR",)

In [ ]:
# make class predictions with the model
predictions = (model.predict(X_test) > 0.98).astype(int)

In [ ]:
for i in range(5):
 print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], y_test[i]))

## Plot Predictions

In [ ]:
# plot all predictions (both signal and background)
plt.figure()
plt.hist(predictions, bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
# plot signal and background separately
plt.figure()
plt.hist(predictions[y_test.astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal')
plt.hist(predictions[~(y_test.astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
frac = 799991 / 600000
799991 / frac

In [ ]:
signal, sig_bins = np.histogram(predictions[y_test.astype(bool)],bins=np.linspace(0,1,50))
bkgrnd, back_bins = np.histogram(predictions[~(y_test.astype(bool))],bins=np.linspace(0,1,50))

# sig_sf = 10 * 7.38400e-05
# back_sf = (10 * 363) / frac
lumi = 10 

sig_sf = 0.00123066835 * lumi
back_sf = (4538.75 * lumi) / frac

plt.hist(sig_bins[:-1], sig_bins, weights=sig_sf*signal, histtype='step',color='midnightblue',label='signal')
plt.hist(back_bins[:-1], back_bins, weights=back_sf*bkgrnd, histtype='step',color='firebrick',label='background')

plt.yscale('log')
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)
plt.show()

In [ ]:
corr_signal = signal[-1]
wrong_background = bkgrnd[-1]
print(f'unscaled signal: {corr_signal}')
print(f'unscaled background {wrong_background}')
print()

lumi= 20

no_s = corr_signal * 0.00123066835 * lumi 
no_b = (wrong_background * 4538.75 * lumi)/frac
significance = no_s / np.sqrt(no_s + no_b)
print(f'S: {no_s}')
print(f'B: {no_b}')
print(f'Significance={round(significance, 4)}')

## Dealing with raw score

In [ ]:
prediction_score = (model.predict(X_test)).astype(float)

In [ ]:
# plot all predictions (both signal and background)
plt.figure()
plt.hist(prediction_score, bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
# plot signal and background separately
plt.figure()
plt.hist(prediction_score[y_test.astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal')
plt.hist(prediction_score[~(y_test.astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
signal, sig_bins = np.histogram(prediction_score[y_test.astype(bool)],bins=np.linspace(0,1,50))
bkgrnd, back_bins = np.histogram(prediction_score[~(y_test.astype(bool))],bins=np.linspace(0,1,50))

sig_sf = 10 * 7.38400e-05
back_sf = (10 * 363) 

plt.hist(sig_bins[:-1], sig_bins, weights=sig_sf*signal, histtype='step',color='midnightblue',label='signal')
plt.hist(back_bins[:-1], back_bins, weights=back_sf*bkgrnd, histtype='step',color='firebrick',label='background')

plt.yscale('log')
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)
plt.show()

In [ ]:
# choose score cuts:
cuts = np.linspace(0,1,500)
nsignal = np.zeros(len(cuts))
nbackground = np.zeros(len(cuts))

for i,cut in enumerate(cuts):
    nsignal[i] = len(np.where(prediction_score[y_test.astype(bool)] > cut)[0])
    nbackground[i] = len(np.where(prediction_score[~(y_test.astype(bool))] > cut)[0])


# plot efficiency vs. purity (ROC curve)
plt.figure()
plt.plot(nsignal/len(X_test[y_test.astype(bool) == 1]),nsignal/(nsignal + nbackground),'o-',color='blueviolet')
# make the plot readable
plt.xlabel('Efficiency',fontsize=12)
plt.ylabel('Purity',fontsize=12)

In [ ]:
# Zoom in view of the upper left corner.
plt.figure()
plt.xlim(0.85, 1.0)
plt.ylim(0.85, 1.0)
# plt.plot([0, 1], [0, 1], 'k--')
plt.plot(nsignal/len(X_test[y_test.astype(bool) == 1]),nsignal/(nsignal + nbackground),'o-',color='blueviolet', markersize=1)
plt.xlabel('Efficiency',fontsize=12)
plt.ylabel('Purity',fontsize=12)
plt.title('PRC(zoomed in at top right)')
plt.show()

## Apply model to gluon set

In [ ]:
mask = X_test[:, -5] > 1.7
X_test_glu = X_test[mask]
y_test_glu = y_test[mask]

In [ ]:
predictions_glu = (model.predict(X_test_glu) > 0.98).astype(int)

In [ ]:
plt.figure()
plt.hist(predictions_glu, bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
# plot signal and background separately
plt.figure()
plt.hist(predictions_glu[y_test_glu.astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal')
plt.hist(predictions_glu[~(y_test_glu.astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background')
# make the plot readable
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
signal_glu, sig_bins_glu = np.histogram(predictions_glu[y_test_glu.astype(bool)],bins=np.linspace(0,1,50))
bkgrnd_glu, back_bins_glu = np.histogram(predictions_glu[~(y_test_glu.astype(bool))],bins=np.linspace(0,1,50))

# sig_sf = 10 * 7.38400e-05
# back_sf = (10 * 363) / frac
lumi = 10 

sig_sf = 0.00123066835 * lumi
back_sf = (4538.75 * lumi) / frac

plt.hist(sig_bins_glu[:-1], sig_bins_glu, weights=sig_sf*signal_glu, histtype='step',color='midnightblue',label='signal_glu')
plt.hist(back_bins_glu[:-1], back_bins_glu, weights=back_sf*bkgrnd_glu, histtype='step',color='firebrick',label='background')

plt.yscale('log')
plt.xlabel('Prediction from NN',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)
plt.show()

In [ ]:
# corr_signal_glu = signal_glu[-1]
# wrong_background_glu = bkgrnd_glu[-1]

corr_signal_glu = 7514
wrong_background_glu = 3

print(f'unscaled signal: {corr_signal_glu}')
print(f'unscaled background {wrong_background_glu}')
print()

lumi= 1100

no_s_glu = corr_signal_glu * 0.00123066835 * lumi 
no_b_glu = (wrong_background_glu * 4538.75 * lumi)/frac
significance_glu = no_s_glu / np.sqrt(no_s_glu + no_b_glu)
print(f'S: {no_s_glu}')
print(f'B: {no_b_glu}')
print(f'Significance={round(significance_glu, 4)}')